**68. Dataframe Joins and column name ambiguity**

In [0]:
orders_list = [("01", "02", 350, 1),
                ("01", "04", 580, 1),
                ("01", "07", 320, 2),
                ("02", "03", 450, 1),
                ("02", "06", 220, 1),
                ("03", "01", 195, 1),
                ("04", "09", 270, 3),
                ("04", "08", 410, 2),
                ("05", "02", 350, 1)]

order_df = spark.createDataFrame(orders_list).toDF("order_id", "prod_id", "unit_price", "qty")

product_list = [("01", "Scroll Mouse", 250, 20),
                ("02", "Optical Mouse", 350, 20),
                ("03", "Wireless Mouse", 450, 50),
                ("04", "Wireless Keyboard", 580, 50),
                ("05", "Standard Keyboard", 360, 10),
                ("06", "16 GB Flash Storage", 240, 100),
                ("07", "32 GB Flash Storage", 320, 50),
                ("08", "64 GB Flash Storage", 430, 25)]

product_df = spark.createDataFrame(product_list).toDF("prod_id", "prod_name", "list_price", "qty")

product_df.show()
order_df.show()

+-------+-------------------+----------+---+
|prod_id|          prod_name|list_price|qty|
+-------+-------------------+----------+---+
|     01|       Scroll Mouse|       250| 20|
|     02|      Optical Mouse|       350| 20|
|     03|     Wireless Mouse|       450| 50|
|     04|  Wireless Keyboard|       580| 50|
|     05|  Standard Keyboard|       360| 10|
|     06|16 GB Flash Storage|       240|100|
|     07|32 GB Flash Storage|       320| 50|
|     08|64 GB Flash Storage|       430| 25|
+-------+-------------------+----------+---+

+--------+-------+----------+---+
|order_id|prod_id|unit_price|qty|
+--------+-------+----------+---+
|      01|     02|       350|  1|
|      01|     04|       580|  1|
|      01|     07|       320|  2|
|      02|     03|       450|  1|
|      02|     06|       220|  1|
|      03|     01|       195|  1|
|      04|     09|       270|  3|
|      04|     08|       410|  2|
|      05|     02|       350|  1|
+--------+-------+----------+---+



In [0]:
join_expr = order_df.prod_id == product_df.prod_id

order_df.join(product_df, join_expr, "inner") \
        .show()

+--------+-------+----------+---+-------+-------------------+----------+---+
|order_id|prod_id|unit_price|qty|prod_id|          prod_name|list_price|qty|
+--------+-------+----------+---+-------+-------------------+----------+---+
|      03|     01|       195|  1|     01|       Scroll Mouse|       250| 20|
|      01|     02|       350|  1|     02|      Optical Mouse|       350| 20|
|      05|     02|       350|  1|     02|      Optical Mouse|       350| 20|
|      02|     03|       450|  1|     03|     Wireless Mouse|       450| 50|
|      01|     04|       580|  1|     04|  Wireless Keyboard|       580| 50|
|      02|     06|       220|  1|     06|16 GB Flash Storage|       240|100|
|      01|     07|       320|  2|     07|32 GB Flash Storage|       320| 50|
|      04|     08|       410|  2|     08|64 GB Flash Storage|       430| 25|
+--------+-------+----------+---+-------+-------------------+----------+---+



In [0]:
product_renamed_df = product_df.withColumnRenamed("qty","reorder_qty")

order_df.join(product_renamed_df, join_expr, "inner") \
        .select("order_id", "prod_name", "unit_price", "qty") \
        .show()

+--------+-------------------+----------+---+
|order_id|          prod_name|unit_price|qty|
+--------+-------------------+----------+---+
|      03|       Scroll Mouse|       195|  1|
|      01|      Optical Mouse|       350|  1|
|      05|      Optical Mouse|       350|  1|
|      02|     Wireless Mouse|       450|  1|
|      01|  Wireless Keyboard|       580|  1|
|      02|16 GB Flash Storage|       220|  1|
|      01|32 GB Flash Storage|       320|  2|
|      04|64 GB Flash Storage|       410|  2|
+--------+-------------------+----------+---+



In [0]:
order_df.join(product_renamed_df, join_expr, "inner") \
        .drop(product_renamed_df.prod_id) \
        .select("order_id", "prod_id", "unit_price", "qty") \
        .show()

+--------+-------+----------+---+
|order_id|prod_id|unit_price|qty|
+--------+-------+----------+---+
|      03|     01|       195|  1|
|      01|     02|       350|  1|
|      05|     02|       350|  1|
|      02|     03|       450|  1|
|      01|     04|       580|  1|
|      02|     06|       220|  1|
|      01|     07|       320|  2|
|      04|     08|       410|  2|
+--------+-------+----------+---+



**69. Outer Joins and Dataframe**

In [0]:
order_df.join(product_renamed_df, join_expr, "outer") \
        .drop(product_renamed_df.prod_id) \
        .select("*") \
        .sort("order_id") \
        .show()

+--------+-------+----------+----+-------------------+----------+-----------+
|order_id|prod_id|unit_price| qty|          prod_name|list_price|reorder_qty|
+--------+-------+----------+----+-------------------+----------+-----------+
|    null|   null|      null|null|  Standard Keyboard|       360|         10|
|      01|     02|       350|   1|      Optical Mouse|       350|         20|
|      01|     04|       580|   1|  Wireless Keyboard|       580|         50|
|      01|     07|       320|   2|32 GB Flash Storage|       320|         50|
|      02|     03|       450|   1|     Wireless Mouse|       450|         50|
|      02|     06|       220|   1|16 GB Flash Storage|       240|        100|
|      03|     01|       195|   1|       Scroll Mouse|       250|         20|
|      04|     08|       410|   2|64 GB Flash Storage|       430|         25|
|      04|     09|       270|   3|               null|      null|       null|
|      05|     02|       350|   1|      Optical Mouse|       350

In [0]:
order_df.join(product_renamed_df, join_expr, "left") \
        .drop(product_renamed_df.prod_id) \
        .select("order_id","prod_id","prod_name", "unit_price","list_price","qty") \
        .sort("order_id") \
        .show()

+--------+-------+-------------------+----------+----------+---+
|order_id|prod_id|          prod_name|unit_price|list_price|qty|
+--------+-------+-------------------+----------+----------+---+
|      01|     04|  Wireless Keyboard|       580|       580|  1|
|      01|     07|32 GB Flash Storage|       320|       320|  2|
|      01|     02|      Optical Mouse|       350|       350|  1|
|      02|     06|16 GB Flash Storage|       220|       240|  1|
|      02|     03|     Wireless Mouse|       450|       450|  1|
|      03|     01|       Scroll Mouse|       195|       250|  1|
|      04|     09|               null|       270|      null|  3|
|      04|     08|64 GB Flash Storage|       410|       430|  2|
|      05|     02|      Optical Mouse|       350|       350|  1|
+--------+-------+-------------------+----------+----------+---+



In [0]:
from pyspark.sql.functions import expr

order_df.join(product_renamed_df, join_expr, "left") \
        .drop(product_renamed_df.prod_id) \
        .select("order_id","prod_id","prod_name", "unit_price","list_price","qty") \
        .withColumn("prod_name",expr("coalesce(prod_name, prod_id)")) \
        .withColumn("list_price",expr("coalesce(list_price, unit_price)")) \
        .sort("order_id") \
        .show()

+--------+-------+-------------------+----------+----------+---+
|order_id|prod_id|          prod_name|unit_price|list_price|qty|
+--------+-------+-------------------+----------+----------+---+
|      01|     04|  Wireless Keyboard|       580|       580|  1|
|      01|     02|      Optical Mouse|       350|       350|  1|
|      01|     07|32 GB Flash Storage|       320|       320|  2|
|      02|     06|16 GB Flash Storage|       220|       240|  1|
|      02|     03|     Wireless Mouse|       450|       450|  1|
|      03|     01|       Scroll Mouse|       195|       250|  1|
|      04|     08|64 GB Flash Storage|       410|       430|  2|
|      04|     09|                 09|       270|       270|  3|
|      05|     02|      Optical Mouse|       350|       350|  1|
+--------+-------+-------------------+----------+----------+---+

